# ROI parameter fitting
Copyright (c) 2021-, Paul Voit
Creative Commons Attribution 4.0 International License
12 May 2022, voit@uni-potsdam.de


The parameters are calculated according to Burn (1990). For every cell we defined the surounding pixels in a 19 x 19 area as neighbors. The user has to specify a weighing function which defines how much the influence of neighboring decreases with increasing distance. We used follwoing formula: $$mu=1-(\frac{d}{26})$$ with *d* as the distance to be weighted.
The parameters for the event ("data/roi_parameters") were calculated with the following script. The only difference is, that because of the size of the region of interest values on the sides (first and last 9 rows, first and last 9 columns) are not identical. This is because for the paper the ROI parameters were calcualted for whole Germany. Because were using a subset here, the values on the sides for the ROI are missing (moving window).  Because the process takes some time (about one hour), the actual calculation in the Example-notebook loads the parameters, which are subset from the Germany wide parameter set, and does not calculate them again. The calculation is based on the yearly maxima for each duration ("data/yearmax_2001_2020").
Sometimes, when calculating the Eta-ROI numpy throws a runtime warning. This happens when there is there is the root of a negative number calculated, which should probably not happen. This returns "nan". Because of this and because we can't check the timeseries of every pixel, we think, that the ROI parameter calculation is less reliable than the other methods. These warnings just occur sometimes and can be ignored. Despite these inconsistencies the results achieved with this method are very similar to the results achieved with the other methods.

In [1]:
import numpy as np
import math
import datetime
import pandas
import os
import xarray as xr

In [2]:
def weighting_f(d, n, tp):
    """
    Function to calculate the weights based on the distance to the center of the raster for ROI-method. Center has weight one.
    This approach is based on the paper of Burn [1990]
    :param d: distance to be weighted, float
    :param n: parameter 1, have to be checked so there  won't be negative weights. This depends on the choosen distance
    :param tp: parameter 2, have to be checked so there  won't be negative weights. This depends on the choosen distance
    :return: weight
    """
    mu = 1 - (d / tp) ** n
    return mu

def PWM(x):
    """
    Probability weighted moments for a series of data points. According to Burn (1990)
    :param x: series of data points, float
    :return: tuple of the first three moments of the distribution
    """
    # np number of years, x timeseries at point
    # sort x?? Hosking et al. Yes
    x = np.array(sorted(x))

    moments = []
    for i in range(0, 3):
        p = [((j - 0.35) / len(x)) ** i for j in range(1, len(x) + 1)]
        p = np.array(p)
        mr = (1 / len(x)) * sum(p * x)
        moments.append(mr)

    return moments

def dist(p1, p2):
    """
    Euclidian distance between two 2D points
    :param p1: tuple, array index (2D)
    :param p2: tuple, array index (2D)
    :return: distance array
    """
    dx = p2[0] - p1[0]
    dy = p2[1] - p1[1]
    d = (dy ** 2 + dx ** 2) ** 0.5
    return d


def roi(yearmax_path, roi_size):
    '''
    Calculates the ROI-parameters for each duration
    :param path2yearmax: String
        FIlepath to the ncdf files that contain the yearly maxima for every duration
    :param roi_size: Int
        Window size of the Region of Interest

    :return: dictionary
        Dictionary which contains for each duration another dictionary containing the four parameters for the ROI-method
    '''
    center = roi_size // 2
    files = os.listdir(yearmax_path)
    durations = [int(file[:2]) for file in files ]
    parms_dict = dict.fromkeys(sorted(durations))

    #create the distance array
    distance_array = np.zeros((roi_size, roi_size))

    for row in range(roi_size):
        for col in range(roi_size):
            distance_array[row, col] = dist([center, center], [row, col])

    for file in files:

        print(f"{datetime.datetime.now()} Fitting ROI parameters for duration {file[:2]} h.")
        nc = xr.open_dataset(f"{yearmax_path}/{file}")
        nc = nc['rainfall_amount'].values
        nc = nc[:20, :, :]

        # pad with nan so no values will be lost. Otherwise np.slinding_window will return a smaller array
        nc = np.pad(nc, center, 'constant', constant_values=np.nan)
        # remove pads for the 3rd(time) dimension
        nc = nc[center:nc.shape[0] - center, :, :]

        #create dictionary to store all the arrays
        parm_names = ["g", "xi", "alpha", "m0"]
        res = dict.fromkeys(parm_names)


        """
        Using this moving window approach one can save the padding when splitting up the array for multiprocessing.
        Basically for each cell in the array there is the window stored as additionally dimensions. The window/cube for
        each respective cell (x,y) can be accessed by: file[:, x, y, :, :]
        """
        nc = np.lib.stride_tricks.sliding_window_view(nc, window_shape=(roi_size, roi_size), axis=(1, 2))

        # arrays to store the results
        g_store = np.empty(shape=(nc.shape[1], nc.shape[2]))
        g_store[:] = np.nan
        xi_store = np.empty(shape=(nc.shape[1], nc.shape[2]))
        xi_store[:] = np.nan
        alpha_store = np.empty(shape=(nc.shape[1], nc.shape[2]))
        alpha_store[:] = np.nan
        m0_store = np.empty(shape=(nc.shape[1], nc.shape[2]))
        m0_store[:] = np.nan

        for row in range(nc.shape[1]):
            for col in range(nc.shape[2]):
                weight_dist = weighting_f(distance_array, 1, 26)
                r = nc[:, row, col, :, :]

                # if np.isnan(r[:, center, center]).sum() != 0:
                if any(np.isnan(r[:, center, center])):
                    continue

                else:
                    pwm_grid = np.zeros((3, roi_size, roi_size))

                    for m in range(0, pwm_grid.shape[1]):
                        for n in range(0, pwm_grid.shape[2]):
                            if any(np.isnan(r[:, m, n])):

                                pwm_grid[0, m, n] = np.nan
                                pwm_grid[1, m, n] = np.nan
                                pwm_grid[2, m, n] = np.nan
                                weight_dist[m, n] = np.nan

                            else:
                                pw_moments = PWM(r[:, m, n])
                                pwm_grid[0, m, n] = pw_moments[0]
                                pwm_grid[1, m, n] = pw_moments[1]
                                pwm_grid[2, m, n] = pw_moments[2]

                    t1 = pwm_grid[1] / pwm_grid[0]
                    t2 = pwm_grid[2] / pwm_grid[0]

                    T1 = (np.nansum(t1 * r.shape[0] * weight_dist)) / (np.nansum(r.shape[0] * weight_dist))
                    T2 = (np.nansum(t2 * r.shape[0] * weight_dist)) / (np.nansum(r.shape[0] * weight_dist))

                    c = (2 * T1 - 1) / (3 * T2 - 1) - math.log(2) / math.log(3)
                    g = 7.859 * c + 2.955 * c ** 2
                    alpha = (2 * T1 - 1) * g / (math.gamma(1 + g) * (1 - 2 ** -g))
                    xi = 1 + alpha * (math.gamma(1 + g) - 1) / g

                    # needed for calculation of return period is also the mean value of r[:,center,center]

                    g_store[row, col] = g
                    xi_store[row, col] = xi
                    alpha_store[row, col] = alpha
                    m0_store[row, col] = r[:, center, center].mean()

        # Writing the parameters to file
        # dummy = pd.DataFrame(g_store)
        # dummy.to_csv(f"roi_g_{file[:2]}.csv", header=False, index=False)
        #
        # dummy = pd.DataFrame(xi_store)
        # dummy.to_csv(f"roi_xi_{file[:2]}.csv", header=False, index=False)
        #
        # dummy = pd.DataFrame(alpha_store)
        # dummy.to_csv(f"roi_alpha_{file[:2]}.csv", header=False, index=False)
        #
        # dummy = pd.DataFrame(m0_store)
        # dummy.to_csv(f"roi_m0_{file[:2]}.csv", header=False, index=False)

        res["g"] = g_store
        res["xi"] = xi_store
        res["alpha"] = alpha_store
        res["m0"] = m0_store

        current_duration = file[:2]
        parms_dict[current_duration] = res

    return parms_dict

In [3]:
# calculate the ROI parameters
roi_parms = roi("data/yearmax_2001_2020", 19)

2022-05-12 09:38:44.493265 Fitting ROI parameters for duration 01 h.
CPU times: user 9min 30s, sys: 16.1 s, total: 9min 47s
Wall time: 9min 29s
